# Step 0 Set up `kgtk`
Use GitHub branch `dev`:

In [ ]:
%%bash
git clone -b dev https://github.com/usc-isi-i2/kgtk.git
cd kgtk
python setup.py install

# Step 1 Send a SPARQL query use `kgtk`

Example 1: Find spouse(s) of politicians: <div>
SPARQL query:
```
SELECT ?politicianLabel ?spouseLabel
WHERE 
{
  ?politician wdt:P31  wd:Q5 ;
              wdt:P106 wd:Q82955 ;
              wdt:P26  ?spouse .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
```

### Define alias and variables

In [ ]:
%%bash
# Database
CLAIMS=../data/claims.tsv.gz
WIKI_INFO=../data/wikidata_infobox.tsv
# Results
RESULTS=../data/spouse_of_politician_kgtk.tsv
NEW_RESULTS=../data/new_spouse_of_politician.tsv
# Datatypes
NUMBERS=../data/numbers.tsv
STRINGS=../data/strings.tsv
STRUCTURED_LITERALS=../data/structued_literals.tsv
QNODES=../data/qnodes.tsv

### Main `kgtk` query:

In [ ]:
%%bash
kgtk query -i $CLAIMS \
           --match 'c: (:Q82955)<-[:P106]-(politician)-[:P26]->(s)' \
           --return 'politician, "P26" as label, s' \
           -o $RESULTS

### Count known results in Wikidata database:

Count politician-spouse pairs / **rows** (result should -1 which is the header):

In [ ]:
%%bash
wc -l $RESULTS

Count how many **unique politicians** have spouse in Wikidata: